# ДЗ № 13, Волжина Лена

Реализуйте алгоритмы Витерби и Forward-Backward для задачи о кривой монете. [Задание](https://compscicenter.ru/learning/assignments/27578/)

![Viterbi](hw13_Viterbi.png)

In [1]:
from collections import defaultdict

class Solver(object):
    def __init__(self, a, pi, b):
        self.a = a
        self.b = b
        self.pi = pi
        self.states = self.a.keys()
    
    def process(self, open_states):
        raise NotImplementedError

In [2]:
class ViterbiSolver(Solver):
    def move_forward(self, open_states):
        n_steps = len(open_states)
        # [t][j] = [step][state]
        deltas, sources = defaultdict(dict), defaultdict(dict)
        
        # init deltas:
        for state in self.states:
            deltas[0][state] = (self.pi[state] * self.b[state][open_states[0]])
        
        # process all the rest open states:
        for step in range(1, n_steps):
            open_state = open_states[step]
            for state in self.states:
                max_prob, max_source = None, None
                for source in self.states:
                    prob = (deltas[step - 1][source] * self.a[source][state] * 
                            self.b[state][open_state])
                    if max_prob is None or max_prob < prob:
                        max_prob, max_source = prob, source
                deltas[step][state] = max_prob
                sources[step][state] = max_source
                
        return deltas, sources
                
    def move_backward(self, deltas, sources):
        n_steps = len(deltas.keys())
        states = [max(self.states, key=deltas[n_steps - 1].get)]
        for step in range(n_steps - 2, -1, -1):
            state = sources[step + 1][states[-1]]
            states.append(state)
        return list(reversed(states))
                
    def process(self, open_states):
        deltas, sources = self.move_forward(open_states)
        states = self.move_backward(deltas, sources)
        return states

In [3]:
a = {'+': {'+': 0.8, '-': 0.2}, '-': {'-': 0.7, '+': 0.3}}
pi = {'+': 0.5, '-': 0.5}
b = {'+': {0: 0.5, 1: 0.5}, '-': {0: 0.9, 1: 0.1}}    # 0 = орел, 1 = решка

open_states = [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]

v_solver = ViterbiSolver(a, pi, b)
print(v_solver.process(open_states))
# v_solver.move_forward(open_states)

['+', '+', '+', '+', '+', '+', '-', '-', '-', '-', '-', '-']


In [4]:
# https://en.wikipedia.org/wiki/Viterbi_algorithm#Example
states = ('Healthy', 'Fever')
observations = ('normal', 'cold', 'dizzy')
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
transition_probability = { 
    'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
    'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
}
emission_probability = {
    'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
}

v_solver1 = ViterbiSolver(transition_probability, start_probability, emission_probability)
print('Result:', v_solver1.process(observations))

Result: ['Healthy', 'Healthy', 'Fever']


# Forward - Backward

In [5]:
class FBSolver(Solver):
    def calculate_alphas_betas(self, open_states):
        n_steps = len(open_states)
        # [t][j] = [step][state]
        alphas, betas = defaultdict(dict), defaultdict(dict)
        
        # init alphas
        for state in self.states:
            alphas[0][state] = (self.pi[state] * self.b[state][open_states[0]])
    
        # calculate alphas
        for step in range(1, n_steps):
            open_state = open_states[step]
            for state in self.states:
                open_prob = self.b[state][open_state]
                prob = sum(alphas[step - 1][source] * self.a[source][state] * open_prob
                           for source in self.states)
                alphas[step][state] = prob
        
        # init betas
        for state in self.states:
            betas[n_steps - 1][state] = 1    # mmm....
        
        # calculate betas
        for step in range(n_steps - 1, 0, -1):
            open_state = open_states[step]         # ошибка в слайдах?
            for state in self.states:
                prob = sum(betas[step][dest] * self.a[state][dest] * self.b[dest][open_state]
                           for dest in self.states)
                betas[step - 1][state] = prob
        
        p_fwd = sum(alphas[n_steps - 1][state] for state in self.states)
        p_bwd = sum(betas[0][state] * self.pi[state] * self.b[state][open_states[0]] for state in self.states)
        print('forward & backward probabilities:', p_fwd, p_bwd)
        return alphas, betas            
    
    def process(self, open_states):
        n_steps = len(open_states)
        alphas, betas = self.calculate_alphas_betas(open_states)
        norm = sum(alphas[n_steps - 1].values())
        result = []
        for step in range(n_steps):
            probs = {}
            for state in self.states:
                probs[state] = alphas[step][state] * betas[step][state] / norm
            result.append(probs)
        
        return result
        

In [6]:
a = {'+': {'+': 0.8, '-': 0.2}, '-': {'-': 0.7, '+': 0.3}}
pi = {'+': 0.5, '-': 0.5}
b = {'+': {0: 0.5, 1: 0.5}, '-': {0: 0.9, 1: 0.1}}    # 0 = орел, 1 = решка

open_states = [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]

fb_solver = FBSolver(a, pi, b)
fb_solver.process(open_states)

forward & backward probabilities: 0.0008441922410745069 0.0008441922410745069


[{'+': 0.5161240679753295, '-': 0.48387593202467044},
 {'+': 0.8241612517632608, '-': 0.17583874823673923},
 {'+': 0.7191711606834265, '-': 0.2808288393165738},
 {'+': 0.8707118848745355, '-': 0.12928811512546454},
 {'+': 0.7150299518329691, '-': 0.2849700481670309},
 {'+': 0.8147187864994426, '-': 0.18528121350055743},
 {'+': 0.47991863117211186, '-': 0.5200813688278881},
 {'+': 0.3346291711940826, '-': 0.6653708288059176},
 {'+': 0.2749522994077331, '-': 0.7250477005922669},
 {'+': 0.2582124217222917, '-': 0.7417875782777082},
 {'+': 0.2724386656062581, '-': 0.7275613343937419},
 {'+': 0.3278043761304334, '-': 0.6721956238695667}]